In [5]:
import rdflib
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF, SDO, RDFS, OWL, DC

import os
from tqdm import tqdm

import sqlparse

In [10]:
global_graph = Graph()
# https://data.mendeley.com/datasets/zp23s23xpb/1
input_folder = "dump/soler/"
for filename in tqdm(os.listdir(input_folder)[:100]):
    f = os.path.join(input_folder, filename)
    try:
        g1 = Graph().parse(f, format='xml')
        global_graph = global_graph + g1
        
    except:
        continue

global_graph.serialize(destination="data/soler.ttl", format="turtle")

100%|██████████| 100/100 [00:09<00:00, 10.34it/s]


<Graph identifier=N8f8b75c9b8674f0ebb50db3fce9db1bb (<class 'rdflib.graph.Graph'>)>

In [9]:
# https://ebiquity.umbc.edu/resource/html/id/82/foafPub-dataset
g = Graph()
g.bind("foaf", FOAF)
g.bind("dc", DC)

with open("dump/foaf_pub/triple_person.sql") as file:
    for i, line in enumerate(tqdm(file)):
        parsed = sqlparse.parse(line)[0]
        try:
            values = str(parsed.tokens[-3])[9:-1].split(',')
            subject = URIRef(values[5][1:-1])
            predicate = URIRef(values[4][1:-1])
            # check if literal
            if values[2] == '1':
                object = Literal(values[3][1:-1])
            else:
                object = URIRef(values[3][1:-1])

                #check uris
                if not (rdflib.term._is_valid_uri(subject) and rdflib.term._is_valid_uri(object)):
                    continue
            
            g.add((subject, predicate, object))
        except:
            continue

        if i > 1000:
            break

g.serialize(destination="data/foaf_pub.ttl", format="turtle")

1001it [00:05, 189.07it/s]


<Graph identifier=Nbe930150f49c41169bb453831ed29938 (<class 'rdflib.graph.Graph'>)>